In [1]:
import requests
import csv
import os
import urllib
import sys
import importlib
import pandas as pd
from shapely.geometry import Point, Polygon
import time
import datetime
from datetime import datetime
from time import sleep
import geopandas 

sys.path.insert(1, '../utils')

import utils

In [2]:
s = requests.Session()

In [3]:
importlib.reload(utils)

<module 'utils' from '../utils/utils.py'>

In [4]:
region = 'New York'
extent = 'region'
county_ids = utils.county_ids.get_county_ids(region, extent)
input_date = '2020-05-01'
input_ts = time.mktime(datetime.strptime(input_date, "%Y-%m-%d").timetuple())
xmin, xmax, ymin, ymax = utils.geometry.osm_bounds(region, county_ids, extent, file = False, raw = True)

url = 'https://api.transitfeeds.com'

In [5]:
# bounding box

coords = [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]
poly = Polygon(coords)

In [6]:
# api key

#key = '9efe3e4d-9b04-467c-9c72-0cc3cf666fd0'
key = 'f2a91a7e-154d-434a-8083-2cd18e25f3d2'

In [7]:
output_feed_info = [list(['operator_name', 'operator_website', 'operator_location', 'operator_metro', 'feed_id', 'date_query', 
                         'date_fetched', 'earliest_calendar_date', 'latest_calendar_date', 'transit_feeds_url'])]

In [8]:
# gets all available locations on transit feeds

response = s.get(
    url+'/v1/getLocations',
    params = {'key': key
       
    }
)
locations = response.json()['results']['locations']
sleep(15)

In [9]:
# filters locations to only those within bounding box

location_ids = []
for operator in locations:
    pt = Point(operator['lng'], operator['lat'])
    if pt.within(poly) is True: 
        location_ids.append(operator['id'])

In [23]:
# loops through all locations
error_count = 0
id_lst = []
for ids in location_ids:
    for attempt in range(4):
        try: 
            response = s.get(
                url+'/v1/getFeeds',
                params = {'key': key, 'location' : ids, 'type': 'gtfs', 'limit': 200

                }
            )
            feeds = response.json()['results']['feeds']
            for agencies in feeds:
                ts = agencies['latest']['ts']
                dt_str = str(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d'))
                name = agencies['t']
                loc = agencies['l']['t']
                dt_fetched = str(datetime.utcfromtimestamp(ts).strftime('%Y%m%d'))
                id_lst.append(list([agencies['id'], dt_fetched, name, loc, dt_str]))
            sleep(1)
            break
        except:
            print(attempt)
            sleep(10)

# check for duplicates

In [24]:
id_lst

[['greater-bridgeport-transit/1022',
  '20191221',
  'Greater Bridgeport Transit GTFS',
  'Bridgeport, CT, USA',
  '2019-12-21'],
 ['mta/81', '20200606', 'MTA Bronx GTFS', 'Bronx, NY, USA', '2020-06-06'],
 ['mta/80',
  '20200605',
  'MTA Brooklyn GTFS',
  'Brooklyn, NY, USA',
  '2020-06-05'],
 ['port-authority-of-new-york-and-new-jersey/384',
  '20200530',
  'PATH GTFS',
  'Jersey City, NJ, USA',
  '2020-05-30'],
 ['suffolk-county-transit/1121',
  '20200116',
  'Suffolk County Transit GTFS',
  'Long Island, New York, USA',
  '2020-01-16'],
 ['mta/86',
  '20200609',
  'Long Island Rail Road GTFS',
  'Long Island, New York, USA',
  '2020-06-09'],
 ['nassau-inter-county-express/268',
  '20170118',
  'NICE GTFS',
  'Nassau, NY, USA',
  '2017-01-18'],
 ['connecticut-transit/322',
  '20170310',
  'CT Transit New Haven GTFS',
  'New Haven, CT, USA',
  '2017-03-10'],
 ['patco/533', '20191211', 'PATCO Speedline', 'New Jersey, USA', '2019-12-11'],
 ['nj-transit/409',
  '20200502',
  'NJ Transit 

In [25]:
gtfs_url = 'https://transitfeeds.com/p/'

In [27]:
for agency in id_lst:
    url = gtfs_url + agency[0] + '/' + agency[1] + '/download'
    urllib.request.urlretrieve(url, "../gtfs/feeds_" + input_date + 
                           "/" + agency[1] + '-' + input_date + ".zip")

NameError: name 'op_name' is not defined

In [ ]:
with open("../gtfs/feeds_" + input_date + "/" + region + "_feed_info_" + input_date + ".csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    for row in id_lst:
        writer.writerow(row)